In [11]:
data_directory="data"
directory="/Users/amaiasagastimartinez/Desktop/SENS-Soundlights/code/sens-sensor"
print(f"Data directory is {data_directory}")
print(f"Working directory is {directory}")

# Set the working directory
import os
os.chdir(directory)

import os
import sys
import wave
import numpy as np
import datetime
import joblib
import warnings
from sklearn.exceptions import InconsistentVersionWarning
# Suppress InconsistentVersionWarning
warnings.filterwarnings("ignore", category=InconsistentVersionWarning)
# 
folder_path="data/listening_test_audios"
seconds_segment=30
model_CLAP_path="data/models/630k-fusion-best_audio_only.pt"
data_path="data"
audio_file_path = "data/MACBA-Edwin/edge-case/2024-12-01_02.wav"

# Load CLAP model
from lib.towers import create_tower
np.random.seed(0)
model_CLAP, _ = create_tower(model_CLAP_path, enable_fusion=True)
print("------- clap model loaded -----------")

# Load PCA model
pca = joblib.load(os.path.join(data_path, "models/pca_model.pkl"))

# Load prediction model (no PCA)
model_e = joblib.load(os.path.join(data_path, "models/models_variations/model_eventfulness_30.joblib"))

# Load prediction model (with PCA)
model_e_pca = joblib.load(os.path.join(data_path, "models/models_variations/model_eventfulness_pca_30.joblib"))

# Time vectors
times_no_pca=[]
times_yes_pca=[]

# Load audio
wf = wave.open(audio_file_path, "rb")
fs = wf.getframerate()
ch = wf.getnchannels()

segment_samples = seconds_segment * fs
print(f"segment_samples {segment_samples}, seconds_segment {seconds_segment}, fs {fs}")
try:
    while True:
        audio_samples = wf.readframes(segment_samples)

        if not audio_samples:
            break  # End of file reached

        # Convert audio_samples to a NumPy array
        audio_samples = np.frombuffer(audio_samples, dtype=np.int16)
        audio_samples = audio_samples.reshape(-1, ch)  # Shape as [time, channels]
        audio_samples = audio_samples[:, 0]  # keep only one channel
        audio_samples = audio_samples / (2**15 - 1)  # 16bit convert to wav [-1,1]

        print(audio_samples.shape)



        ##### No PCA ####
        
        # Extract features
        features_segment = model_CLAP.get_audio_embedding_from_data(
            [audio_samples], use_tensor=False
        )
        features_pca=pca.transform(features_segment)
        # Do prediction
        start_date_time_NO=datetime.datetime.now()
        e_prediction=model_e.predict(features_segment)
        time_difference_NO=(datetime.datetime.now()-start_date_time_NO).total_seconds()
        print("NO PCA ", time_difference_NO)
        


       
    print("-----------------------")
finally:
    wf.close()

Data directory is data
Working directory is /Users/amaiasagastimartinez/Desktop/SENS-Soundlights/code/sens-sensor
------- clap model loaded -----------
segment_samples 1440000, seconds_segment 30, fs 48000
(1440000,)
NO PCA  0.000694
(1440000,)
NO PCA  0.000631
(96000,)
NO PCA  0.000586
-----------------------


In [13]:
data_directory="data"
directory="/Users/amaiasagastimartinez/Desktop/SENS-Soundlights/code/sens-sensor"
print(f"Data directory is {data_directory}")
print(f"Working directory is {directory}")

# Set the working directory
import os
os.chdir(directory)

import os
import sys
import wave
import numpy as np
import datetime
import joblib
# 
folder_path="data/listening_test_audios"
seconds_segment=30
model_CLAP_path="data/models/630k-fusion-best_audio_only.pt"
data_path="data"
audio_file_path = "data/MACBA-Edwin/edge-case/2024-12-01_02.wav"

# Load CLAP model
from lib.towers import create_tower
np.random.seed(0)
model_CLAP, _ = create_tower(model_CLAP_path, enable_fusion=True)
print("------- clap model loaded -----------")

# Load PCA model
pca = joblib.load(os.path.join(data_path, "models/pca_model.pkl"))

# Load prediction model (no PCA)
model_e = joblib.load(os.path.join(data_path, "models/models_variations/model_eventfulness_30.joblib"))

# Load prediction model (with PCA)
model_e_pca = joblib.load(os.path.join(data_path, "models/models_variations/model_eventfulness_pca_30.joblib"))

# Time vectors
times_no_pca=[]
times_yes_pca=[]

# Load audio
wf = wave.open(audio_file_path, "rb")
fs = wf.getframerate()
ch = wf.getnchannels()

segment_samples = seconds_segment * fs
print(f"segment_samples {segment_samples}, seconds_segment {seconds_segment}, fs {fs}")
try:
    while True:
        audio_samples = wf.readframes(segment_samples)

        if not audio_samples:
            break  # End of file reached

        # Convert audio_samples to a NumPy array
        audio_samples = np.frombuffer(audio_samples, dtype=np.int16)
        audio_samples = audio_samples.reshape(-1, ch)  # Shape as [time, channels]
        audio_samples = audio_samples[:, 0]  # keep only one channel
        audio_samples = audio_samples / (2**15 - 1)  # 16bit convert to wav [-1,1]

        print(audio_samples.shape)

        ##### YES PCA ####
        # Extract features
        features_segment = model_CLAP.get_audio_embedding_from_data(
            [audio_samples], use_tensor=False
        )
        # Apply PCA
        features_pca=pca.transform(features_segment)
        # Do prediction
        start_date_time_YES=datetime.datetime.now()
        e_prediction_pca=model_e_pca.predict(features_pca)
        time_difference_YES=(datetime.datetime.now()-start_date_time_YES).total_seconds()
        
        print("YES PCA ", time_difference_YES)

       
    print("-----------------------")
finally:
    wf.close()

Data directory is data
Working directory is /Users/amaiasagastimartinez/Desktop/SENS-Soundlights/code/sens-sensor
------- clap model loaded -----------
segment_samples 1440000, seconds_segment 30, fs 48000
(1440000,)
YES PCA  0.000628
(1440000,)
YES PCA  0.000615
(96000,)
YES PCA  0.000604
-----------------------


In [5]:
print("NO PCA ", times_no_pca)
print("-----")
print("With PCA ", times_yes_pca)

NO PCA  []
-----
With PCA  []
